This notebook generates the following panels:
- ( ) Summary: distribution of depth preferences of all cells

use env 2p_analysis_cottage

In [ ]:
VERSION = 1

In [ ]:
%reload_ext autoreload
%autoreload 2

import os
import sys
import defopt
import pickle
import numpy as np
import pandas as pd
import scipy.stats
from scipy.optimize import curve_fit
import matplotlib
# import matplotlib as mpl
from matplotlib import cm
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
mpl.rcParams['pdf.fonttype'] = 42 # save text as text not outlines
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.colors import ListedColormap
# from mpl_toolkits.axes_grid1 import make_axes_locatable
import warnings

from cottage_analysis.depth_analysis.filepath import generate_filepaths
from cottage_analysis.stimulus_structure.sphere_structure import *
from cottage_analysis.depth_analysis.depth_preprocess.process_params import create_speed_arr, create_trace_arr_per_roi, calculate_OF, thr
from cottage_analysis.depth_analysis.plotting.plotting_utils import *
from cottage_analysis.depth_analysis.plotting.basic_vis_plots import *

In [ ]:
def get_depth_color(depth, depth_list, cmap=cm.RdYlBu):
    norm = matplotlib.colors.Normalize(vmin=np.log(min(depth_list)), vmax=np.log(max(depth_list)))
    rgba_color = cmap(norm(np.log(depth)),bytes=True)
    rgba_color = tuple(it/255 for it in rgba_color)
    
    return rgba_color


def plot_spatial_map(depth_neurons, non_depth_neurons, stat, im_background, depth_list, cmap):
    for n in non_depth_neurons:
        ypix = stat[n]['ypix'][~stat[n]['overlap']]
        xpix = stat[n]['xpix'][~stat[n]['overlap']]
        if len(xpix) > 0 and len(ypix) > 0:
            im_background[ypix,xpix,:] = np.tile((stat[n]['lam'][~stat[n]['overlap']])/np.max(stat[n]['lam'][~stat[n]['overlap']]), (3,1)).T

    for i, n in enumerate(depth_neurons):
        ypix = stat[n]['ypix'][~stat[n]['overlap']]
        xpix = stat[n]['xpix'][~stat[n]['overlap']]
        lam_mat = np.tile((stat[n]['lam'][~stat[n]['overlap']])/np.max(stat[n]['lam'][~stat[n]['overlap']]), (3,1)).T
        max_depth = np.exp(max_depths[i])
        rgba_color = get_depth_color(depth=max_depth, depth_list=depth_list, cmap=cmap)
        im_background[ypix,xpix,:] =((np.asarray(rgba_color)[:-1].reshape(-1,1))@(lam_mat[:,0].reshape(1,-1))).T

    plt.figure(figsize=(10,10))
    # plt.imshow(ops['meanImg'], cmap='gray', alpha=1) # change vmax
    plt.imshow(im_background[:,20:,:], alpha=1) 
    plt.axis('off')

In [ ]:
# Load data
root = '/camp/lab/znamenskiyp/home/shared/presentations/PetrTalk202302/Spatial_distribution_map/'
ops = np.load(root + 'data/ops.npy', allow_pickle=True)
ops = ops.item()
iscell = np.load(root + 'data/iscell.npy', allow_pickle=True)[:, 0]
which_rois = (np.arange(iscell.shape[0]))[iscell.astype('bool')]
stat = np.load(root + 'data/stat.npy',allow_pickle=True)
with open(root+'data/gaussian_depth_tuning_fit_new_0.5.pickle','rb') as handle:
    max_depths_df = pickle.load(handle)
max_depths = max_depths_df.x0_logged.values
depth_neurons = np.load(root + 'data/depth_neurons.npy')
depth_list = [6, 19, 60, 190, 600]

In [ ]:
# Depth neurons coded in continuous colours
cmap = cm.cool.reversed()
background_color = np.array([0.133,0.545,0.133])
im = np.zeros((ops['Ly'], ops['Lx'],3))  # replace this with background

im_back = np.swapaxes(np.swapaxes(np.tile(ops['meanImg'],(3,1,1)),0,2),0,1)/np.max(ops['meanImg'])*3
im_back = np.multiply(im_back,background_color.reshape(1,-1))

non_depth_neurons = [x for x in which_rois if x not in depth_neurons]
    
plot_spatial_map(depth_neurons=depth_neurons, 
                 non_depth_neurons=non_depth_neurons, 
                 stat=stat, 
                 im_background=im, 
                 depth_list=depth_list, 
                 cmap=cmap)
save_prefix = 'plots/'
if not os.path.exists(root+save_prefix):
    os.makedirs(root+save_prefix)
plt.savefig(root+save_prefix+'map_continuous_no_background.pdf', dpi=1000)

plot_spatial_map(depth_neurons=depth_neurons, 
                 non_depth_neurons=non_depth_neurons, 
                 stat=stat, 
                 im_background=im_back, 
                 depth_list=depth_list, 
                 cmap=cmap)

plt.savefig(root+save_prefix+'map_continuous_meanImg.pdf', dpi=1000)

In [ ]:
# ----- SETUPS -----
project = 'hey2_3d-vision_foodres_20220101'
mouse = 'PZAH6.4b'
session = 'S20220426'
protocol = 'SpheresPermTubeReward'

rawdata_root = '/camp/lab/znamenskiyp/data/instruments/raw_data/projects/'
root = '/camp/lab/znamenskiyp/home/shared/projects/'
depth_list = [0.06, 0.19, 0.6, 1.9, 6]
choose_trials = 50
frame_rate = 15
speed_thr_cal = 0.2  # m/s, threshold for running speed when calculating depth neurons
speed_thr = 0.01  #m/s

# ----- STEP1: Generate file path -----
print('---START STEP 1---', '\n', 'Getting data filepaths...', flush=True)
rawdata_folder, protocol_folder, analysis_folder, suite2p_folder, trace_folder = generate_filepaths.generate_file_folders(
    project=project,
    mouse=mouse,
    session=session,
    protocol=protocol,
    rawdata_root=rawdata_root,
    root=root)
print(rawdata_folder, protocol_folder, analysis_folder, suite2p_folder, trace_folder, flush=True)

print('---STEP 1 FINISHED.---', '\n', flush=True)


# ----- STEP2: Load files -----
print('---START STEP 2---', '\n', 'Load files...', flush=True)
# Load suite2p files
ops = np.load(suite2p_folder/'ops.npy', allow_pickle=True)
ops = ops.item()
stat = np.load(suite2p_folder/'stat.npy', allow_pickle=True)
iscell = np.load(suite2p_folder/'iscell.npy', allow_pickle=True)[:, 0]
F = np.load(trace_folder/'F.npy', allow_pickle=True)
Fast = np.load(trace_folder/'Fast.npy', allow_pickle=True)
# Fneu = np.load(trace_folder + 'Fneu.npy', allow_pickle=True)
# spks = np.load(trace_folder + 'spks.npy', allow_pickle=True)

# All_rois
which_rois = (np.arange(F.shape[0]))[iscell.astype('bool')]

# dffs_ast
dffs = np.load(trace_folder/'dffs_ast.npy')

# Load depth neurons
depth_neurons = np.load(analysis_folder/'plane0/depth_neurons.npy')
max_depths = np.load(analysis_folder/'plane0/max_depths_index.npy')
max_depths_gaussian_fit = np.load(analysis_folder/'plane0/max_LogDepth_gaussianfit.npy')
print('---STEP 2 FINISHED.---', '\n', flush=True)

# Depth neurons coded in continuous colours
def get_depth_color(depth, depth_list, cmap=cm.RdYlBu):
    norm = matplotlib.colors.Normalize(vmin=np.log(min(depth_list)), vmax=np.log(max(depth_list)))
    rgba_color = cmap(norm(np.log(depth)),bytes=True)
    rgba_color = tuple(it/255 for it in rgba_color)
    
    return rgba_color

cmap = cm.cool.reversed()
background_color = np.array([0.133,0.545,0.133])
im = np.zeros((ops['Ly'], ops['Lx'],3))  # replace this with background

im_back = np.swapaxes(np.swapaxes(np.tile(ops['meanImg'],(3,1,1)),0,2),0,1)/np.max(ops['meanImg'])
im_back = np.multiply(im_back,background_color.reshape(1,-1))
im = im_back

non_depth_neurons = [x for x in which_rois if x not in depth_neurons]
depth_neurons_plot = [x for x in depth_neurons if x not in \
                      [789,2356,1810,1216, 676, 502, 667, 1060, 216, 994, 265, 366, 1742, 909, 1842, 133, 918, 1060, 1742, 1369, 1378, 1499, 911]]
for n in non_depth_neurons:
    ypix = stat[n]['ypix'][~stat[n]['overlap']]
    xpix = stat[n]['xpix'][~stat[n]['overlap']]
    if len(xpix) > 0 and len(ypix) > 0:
        im[ypix,xpix,:] = np.tile((stat[n]['lam'][~stat[n]['overlap']])/np.max(stat[n]['lam'][~stat[n]['overlap']]), (3,1)).T

for i, n in enumerate(depth_neurons_plot):
    ypix = stat[n]['ypix'][~stat[n]['overlap']]
    xpix = stat[n]['xpix'][~stat[n]['overlap']]
    lam_mat = np.tile((stat[n]['lam'][~stat[n]['overlap']])/np.max(stat[n]['lam'][~stat[n]['overlap']]), (3,1)).T
    max_depth = np.exp(max_depths_gaussian_fit[i,1])
    rgba_color = get_depth_color(depth=max_depth, depth_list=depth_list, cmap=cmap)
    im[ypix,xpix,:] =((np.asarray(rgba_color)[:-1].reshape(-1,1))@(lam_mat[:,0].reshape(1,-1))).T

plt.figure(figsize=(10,10))
# plt.imshow(ops['meanImg'], cmap='gray', alpha=1) # change vmax
plt.imshow(im[20:,20:,:], alpha=1) 
plt.axis('off')
save_root = '/camp/lab/znamenskiyp/home/shared/presentations/Cosyne2023/ver'+str(VERSION)+'/example_spatial_map/'
if not os.path.exists(save_root):
    os.makedirs(save_root)
plt.savefig(f'{save_root}map_continuous_{mouse}_{session}.pdf', dpi=1000)

# plt.figure(figsize=(10,10))
# # plt.imshow(ops['meanImg'], cmap='gray', alpha=1) # change vmax
# plt.imshow(im_back[:,20:,:], alpha=1) 
# plt.axis('off')
# plt.savefig(analysis_folder/save_prefix/'meanimg.pdf', dpi=1000)
